In [1]:
%pip install pandas
%pip install sodapy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [15]:
# Intake all rows from API and store them in a pandas DataFrame
#
# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

api_key=""

# Create a client to retrieve data from Socrata API
client = Socrata("data.lacity.org", api_key)

# parameters for API call
limit = 2000
offset = 0
all_data=[]

# Iterations of total limit of rows until the end of the data set, returned as JSON from API
# converted to Python list of dictionaries by sodapy.
while True:
    try:
        results = client.get("nqb5-fsih", limit=limit, offset=offset)

        if not results:
            print ("End of dataset reached")
            break

        all_data.extend(results)
        offset += limit

        print(f"Fetched {len(results)} rows. (total so far: {len(all_data)})")

    except Exception as e:
        print(f"Error occured: {e}")
        break
        
print(f"Total rows downloaded: {len(all_data)}")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(all_data)
#print (results_df)

Fetched 2000 rows. (total so far: 2000)
Fetched 2000 rows. (total so far: 4000)
Fetched 2000 rows. (total so far: 6000)
Fetched 2000 rows. (total so far: 8000)
Fetched 146 rows. (total so far: 8146)
End of dataset reached
Total rows downloaded: 8146


In [47]:
# Extract GEOJSON format coordinates from dataset
# At the time of upload, the API was updated with separate longitude and latitude columns

def extract_coordinates(row):
    if isinstance(row,dict) and 'coordinates' in row:
        coords = row['coordinates']
        return coords[0], coords[1]
    return None, None

# Extract the longitude and latitude and add them as columns in results_df
results_df['longitude'], results_df['latitude'] = zip(*results_df['location_1'].apply(extract_coordinates))

#Drop the column with the GEOJSON format
results_df = results_df.drop(columns=['location_1'])

location_account                                             0000000467-0001-5
business_name                LOUISIANA TO LOS ANGELES ORGANIZING COMMITTEE INC
dba_name                                                                  LALA
street_address                                            9461 LA SALLE AVENUE
city                                                               LOS ANGELES
zip_code                                                            90047-3836
location_description                                  9461 LA SALLE 90047-3836
mailing_address                                           9461 LA SALLE AVENUE
mailing_city                                                       LOS ANGELES
mailing_zip_code                                                    90047-3836
naics                                                                   722110
primary_naics_description                             Full-service restaurants
council_district                                    

In [48]:
# Store the cleaned up data into a PostgreSQL table named raw_data in the dataapi database

import pandas as pd
from sqlalchemy import create_engine

#database credentials
db_user= "postgres"
db_pass = ""
db_host = "Postgres"
db_port = "5432"
db_name = "dataapi"


connection_str = f"postgresql://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(connection_str)

try:
    print ("Sending data to PostgreSQL...")

    results_df.to_sql(
        name = 'raw_data',      # name of table
        con = engine,           # connection engine
        if_exists = 'replace',  # Options: 'fail', 'replace', 'append'
        index = False,          # Do not write the DataFrame index as a column
        chunksize = 1000        # Write 1000 rows at a time
    )

    print("Data exported successfully!")

    with engine.connect() as conn:
        result = pd.read_sql("Select count(*) FROM raw_data", conn)
        print(f"  Rows in database: {result.iloc[0,0]}")

except Exception as e:
    print(f"Error: {e}")

Sending data to PostgreSQL...
Data exported successfully!
  Rows in database: 8146
